In [23]:
import pandas as pd

## Importing the Dataset
#### We will first import the dataset and perform data cleaning steps

In [24]:
df = pd.read_csv('spam.csv',encoding="ISO-8859-1")

In [25]:
df[:5]

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [26]:
df.shape

(5572, 5)

In [27]:
df.columns = ['label', 'sms1', 'sms2', 'sms3', 'sms4']

In [28]:
df[:5]

,label,sms1,sms2,sms3,sms4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [29]:
df.isnull().sum(axis = 0)

label       0
sms1        0
sms2     5522
sms3     5560
sms4     5566
dtype: int64

In [30]:
##Dropping columns as majority of the values in these columns is null
df = df.drop(['sms2','sms3','sms4'], 1)

C:\Users\cdeep\AppData\Local\Temp\ipykernel_13496\97667530.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df = df.drop(['sms2','sms3','sms4'], 1)


In [31]:
df

,label,sms1
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will Ì_ b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


## Text Preprocessing

In [32]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\cdeep\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [33]:
ps = PorterStemmer()

In [34]:
corpus = []

## Text preperation
###   
####    1. First we are we are removing all the character except a-z and A-Z and replacing them with  SPACE.
####    2. Then we are converting the text data to lower case because CountVectorizer will consider the    same word as seperate words if they start with small and capital letter.
####    3. Then we apply stemming.

In [35]:
for i in range(len(df)):
    review = re.sub('[^a-zA-Z]', ' ', df['sms1'][i])
    review = review.lower()
    review = review.split()
    review = [ps.stem(word) for word in review if not word in set(stopwords.words('english'))]
    review = ' '.join(review)
    corpus.append(review)

In [36]:
corpus[:5]

['go jurong point crazi avail bugi n great world la e buffet cine got amor wat',
 'ok lar joke wif u oni',
 'free entri wkli comp win fa cup final tkt st may text fa receiv entri question std txt rate c appli',
 'u dun say earli hor u c alreadi say',
 'nah think goe usf live around though']

In [37]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=2500)
X = cv.fit_transform(corpus).toarray()

In [38]:
y=pd.get_dummies(df['label'], drop_first = True)

## Creating Training and Testing Data and training models

In [39]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y.values.ravel(), test_size = 0.20, random_state = 0)
from sklearn.naive_bayes import MultinomialNB
spam_detect_model = MultinomialNB().fit(X_train, y_train)

In [40]:
y_pred=spam_detect_model.predict(X_test)

In [41]:
from sklearn.metrics import confusion_matrix
confusion_m = confusion_matrix(y_test, y_pred)
confusion_m

array([[943,   6],
       [  9, 157]], dtype=int64)

In [42]:
from sklearn.ensemble import RandomForestClassifier

classifier = RandomForestClassifier()
classifier.fit(X_train, y_train)

y_pred2 = classifier.predict(X_test)
cm = confusion_matrix(y_test, y_pred2)
cm

array([[948,   1],
       [ 25, 141]], dtype=int64)

In [43]:
from sklearn.metrics import accuracy_score

In [44]:
print('naive_bayes : ',accuracy_score(y_test, y_pred))
print('RandomForest : ',accuracy_score(y_test, y_pred2))

naive_bayes :  0.9865470852017937
RandomForest :  0.9766816143497757


#### As we can see naive bayes outperforms the randomforest model so our final model will be spam_detect_model (naive bayes model)